# Merge Excel Files for Attraction Data
This notebook merges multiple Excel files containing attraction, transaction, and location data into a single dataset with the specified format.

Required output format includes 21 columns: TransactionId, UserId, VisitYear, VisitMonth, VisitMode, AttractionId, Rating, AttractionCityId, AttractionTypeId, Attraction, AttractionAddress, AttractionType, ContenentId, RegionId, CountryId, CityId, CityName, Contenent, Country, Region, Overall_Avg_Rating.

Ensures VisitMode is populated by retaining Transaction.xlsx values or recovering from Mode.xlsx if null.

In [1]:
# Import required libraries
import pandas as pd
import os

In [2]:
# Define file paths and output
input_directory = r'D:\Project_Tourism_Experience_Analytics\Tourism Dataset\Dataset_merging'  # Update with your directory path
output_file = 'merged_attractions_data.xlsx'

# Dictionary of Excel files
files = {
    'Transaction': os.path.join(input_directory, 'Transaction.xlsx'),
    'Mode': os.path.join(input_directory, 'Mode.xlsx'),
    'Item': os.path.join(input_directory, 'Item.xlsx'),
    'Type': os.path.join(input_directory, 'Type.xlsx'),
    'City': os.path.join(input_directory, 'City.xlsx'),
    'Country': os.path.join(input_directory, 'Country.xlsx'),
    'Region': os.path.join(input_directory, 'Region.xlsx'),
    'Continent': os.path.join(input_directory, 'Continent.xlsx'),
    'User': os.path.join(input_directory, 'User.xlsx')
}

In [3]:
# Read all Excel files into DataFrames with error handling
dfs = {}
for key, file_path in files.items():
    try:
        if os.path.exists(file_path):
            dfs[key] = pd.read_excel(file_path)
            print(f'Loaded {key} with shape: {dfs[key].shape}')
        else:
            print(f'File not found: {file_path}')
    except Exception as e:
        print(f'Error loading {key} from {file_path}: {str(e)}')

# Check if all files were loaded
if len(dfs) != len(files):
    print('Not all files were loaded successfully. Please check the errors above.')
else:
    # Start with Transaction data
    merged_df = dfs['Transaction'].copy()

    # Inspect VisitMode in Transaction.xlsx
    print("\nUnique values in Transaction VisitMode:", dfs['Transaction']['VisitMode'].unique())
    print("Null count in Transaction VisitMode:", dfs['Transaction']['VisitMode'].isna().sum())

    # Skip Mode merge (assuming VisitMode in Transaction.xlsx contains names)
    # If VisitMode contains IDs, uncomment the following block and adjust types
    """
    # Ensure VisitMode and VisitModeId are same type (e.g., string)
    merged_df['VisitMode'] = merged_df['VisitMode'].astype(str)
    dfs['Mode']['VisitModeId'] = dfs['Mode']['VisitModeId'].astype(str)
    merged_df = pd.merge(
        merged_df,
        dfs['Mode'][['VisitModeId', 'VisitMode']],
        left_on='VisitMode',
        right_on='VisitModeId',
        how='left'
    ).drop(columns=['VisitModeId'])
    merged_df = merged_df.rename(columns={'VisitMode_y': 'VisitMode'}).drop(columns=['VisitMode_x'], errors='ignore')
    """

    # Join with Item on AttractionId
    merged_df = pd.merge(
        merged_df,
        dfs['Item'][['AttractionId', 'AttractionCityId', 'AttractionTypeId', 'Attraction', 'AttractionAddress']],
        on='AttractionId',
        how='left'
    )

    # Join with Type on AttractionTypeId
    merged_df = pd.merge(
        merged_df,
        dfs['Type'][['AttractionTypeId', 'AttractionType']],
        on='AttractionTypeId',
        how='left'
    )

    # Join with City on AttractionCityId (as CityId)
    merged_df = pd.merge(
        merged_df,
        dfs['City'][['CityId', 'CityName', 'CountryId']],
        left_on='AttractionCityId',
        right_on='CityId',
        how='left'
    ).drop(columns=['CityId'])

    # Join with Country on CountryId
    merged_df = pd.merge(
        merged_df,
        dfs['Country'][['CountryId', 'Country', 'RegionId']],
        on='CountryId',
        how='left'
    )

    # Join with Region on RegionId
    merged_df = pd.merge(
        merged_df,
        dfs['Region'][['RegionId', 'Region', 'ContinentId']],
        on='RegionId',
        how='left'
    )

    # Join with Continent on ContinentId
    merged_df = pd.merge(
        merged_df,
        dfs['Continent'][['ContinentId', 'Continent']],
        on='ContinentId',
        how='left'
    )

    # Join with User on UserId
    merged_df = pd.merge(
        merged_df,
        dfs['User'][['UserId', 'ContinentId', 'RegionId', 'CountryId', 'CityId']],
        on='UserId',
        how='left',
        suffixes=('', '_user')
    )

    # Calculate Overall_Avg_Rating (mean Rating per AttractionId)
    avg_rating = merged_df.groupby('AttractionId')['Rating'].mean().reset_index(name='Overall_Avg_Rating')
    merged_df = pd.merge(
        merged_df,
        avg_rating,
        on='AttractionId',
        how='left'
    )

    # Define required columns (fixed to avoid duplicate VisitMode)
    required_columns = [
        'TransactionId', 'UserId', 'VisitYear', 'VisitMonth', 'VisitMode', 'AttractionId', 'Rating',
        'AttractionCityId', 'AttractionTypeId', 'Attraction', 'AttractionAddress', 'AttractionType',
        'ContinentId', 'RegionId', 'CountryId', 'CityId', 'CityName', 'Continent', 'Country', 'Region',
        'Overall_Avg_Rating'
    ]

    # Ensure all required columns exist
    for col in required_columns:
        if col not in merged_df.columns:
            merged_df[col] = pd.NA

    # Reorder columns and drop extras
    merged_df = merged_df[required_columns]

    # Rename columns to match required format (including typos)
    merged_df = merged_df.rename(columns={
        'ContinentId': 'ContenentId',
        'Continent': 'Contenent'
    })

    # Enforce correct data types
    type_dict = {
        'TransactionId': 'int64',
        'UserId': 'int64',
        'VisitYear': 'int64',
        'VisitMonth': 'int64',
        'VisitMode': 'object',  # String for mode names (e.g., Couples)
        'AttractionId': 'int64',
        'Rating': 'int64',
        'AttractionCityId': 'int64',
        'AttractionTypeId': 'int64',
        'Attraction': 'object',
        'AttractionAddress': 'object',
        'AttractionType': 'object',
        'ContenentId': 'int64',
        'RegionId': 'int64',
        'CountryId': 'int64',
        'CityId': 'int64',
        'CityName': 'object',
        'Contenent': 'object',
        'Country': 'object',
        'Region': 'object',
        'Overall_Avg_Rating': 'float64'
    }
    for col, dtype in type_dict.items():
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].astype(dtype, errors='ignore')

    # Check for unexpected columns (e.g., VisitMode.1)
    unexpected_cols = [col for col in merged_df.columns if col not in type_dict]
    if unexpected_cols:
        print(f"Unexpected columns found: {unexpected_cols}")
        merged_df = merged_df.drop(columns=unexpected_cols)

    # Validate VisitMode
    print("\nNull count in final VisitMode:", merged_df['VisitMode'].isna().sum())
    print("Unique values in final VisitMode:", merged_df['VisitMode'].unique())

    # If VisitMode is still null, attempt to recover from Mode.xlsx
    if merged_df['VisitMode'].isna().all():
        print("\nVisitMode is all null. Attempting to recover using Mode.xlsx...")
        merged_df = merged_df.drop(columns=['VisitMode'])
        # Ensure type consistency
        merged_df['VisitMode_orig'] = dfs['Transaction']['VisitMode'].astype(str)
        dfs['Mode']['VisitModeId'] = dfs['Mode']['VisitModeId'].astype(str)
        merged_df = pd.merge(
            merged_df,
            dfs['Mode'][['VisitModeId', 'VisitMode']],
            left_on='VisitMode_orig',
            right_on='VisitModeId',
            how='left'
        ).drop(columns=['VisitModeId', 'VisitMode_orig'])
        print("Null count after recovery:", merged_df['VisitMode'].isna().sum())

    # Save the merged DataFrame to Excel
    merged_df.to_excel(output_file, index=False)
    print(f'Merged Excel file saved as {output_file}')

    # Display the first few rows and info
    display(merged_df.head())
    merged_df.info()

Loaded Transaction with shape: (52930, 7)
Loaded Mode with shape: (6, 2)
Loaded Item with shape: (1698, 5)
Loaded Type with shape: (17, 2)
Loaded City with shape: (9143, 3)
Loaded Country with shape: (165, 3)
Loaded Region with shape: (22, 3)
Loaded Continent with shape: (6, 2)
Loaded User with shape: (33530, 5)

Unique values in Transaction VisitMode: [2 4 3 5 1]
Null count in Transaction VisitMode: 0

Null count in final VisitMode: 0
Unique values in final VisitMode: [2 4 3 5 1]
Merged Excel file saved as merged_attractions_data.xlsx


,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating,AttractionCityId,AttractionTypeId,Attraction,...,AttractionType,ContenentId,RegionId,CountryId,CityId,CityName,Contenent,Country,Region,Overall_Avg_Rating
0,3,70456,2022,10,2,640,5,1,63,Sacred Monkey Forest Sanctuary,...,Nature & Wildlife Areas,1,1,1,4341.0,Douala,Africa,Cameroon,Central Africa,4.267086
1,8,7567,2022,10,4,640,5,1,63,Sacred Monkey Forest Sanctuary,...,Nature & Wildlife Areas,1,1,1,464.0,Douala,Africa,Cameroon,Central Africa,4.267086
2,9,79069,2022,10,3,640,5,1,63,Sacred Monkey Forest Sanctuary,...,Nature & Wildlife Areas,1,1,1,774.0,Douala,Africa,Cameroon,Central Africa,4.267086
3,10,31019,2022,10,3,640,3,1,63,Sacred Monkey Forest Sanctuary,...,Nature & Wildlife Areas,1,1,1,583.0,Douala,Africa,Cameroon,Central Africa,4.267086
4,15,43611,2022,10,2,640,3,1,63,Sacred Monkey Forest Sanctuary,...,Nature & Wildlife Areas,1,1,1,1396.0,Douala,Africa,Cameroon,Central Africa,4.267086


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52930 entries, 0 to 52929
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TransactionId       52930 non-null  int64  
 1   UserId              52930 non-null  int64  
 2   VisitYear           52930 non-null  int64  
 3   VisitMonth          52930 non-null  int64  
 4   VisitMode           52930 non-null  object 
 5   AttractionId        52930 non-null  int64  
 6   Rating              52930 non-null  int64  
 7   AttractionCityId    52930 non-null  int64  
 8   AttractionTypeId    52930 non-null  int64  
 9   Attraction          52930 non-null  object 
 10  AttractionAddress   52930 non-null  object 
 11  AttractionType      52930 non-null  object 
 12  ContenentId         52930 non-null  int64  
 13  RegionId            52930 non-null  int64  
 14  CountryId           52930 non-null  int64  
 15  CityId              52922 non-null  float64
 16  City